In [ ]:
import numpy
import os
import pandas as pd
import pickle
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.inspection import permutation_importance
from scipy.cluster.hierarchy import dendrogram
from matplotlib import pyplot as plt



Most if not all of the files here will be contained in /media/data/BIAS/microbiome_GCN/AGES/

Note that if feature_importance has already been computed, then you can skip directly to the bottom where you load it with pickle.\
\
Also note that "miniproject-feature-significance.py" can be directly run to do everything at once.

First we load all the prerequisite files, process them and also grab some of information we will need down the line.

In [ ]:
df_meta = pd.read_csv("/media/data/BIAS/microbiome_GCN/AGES/AGP_ages.metadata.txt", sep='\t')
df_meta = df_meta.set_index("#SampleID")

labels = df_meta['age_cat']  # the classes for the variable age_cat are the following: ['20s','30s','40s','50s','60s'] 
Sample_ids = labels.keys()

df_AGES = pd.read_csv("/media/data/BIAS/microbiome_GCN/AGES/AGP.data.biom.filtered.ages.tsv",sep='\t')
features = df_AGES["#OTU ID"]
df_AGES = df_AGES.set_index("#OTU ID")

loading the reference phylogeny which we will use to identify the RELATIONSHIP between the abundance of the different microbes (sequences)

In [ ]:
# gg_phylogeny = TreeNode.read("../../2024.09.phylogeny.asv.nwk") #for now you don't actually need to load it

In [ ]:
dist_dict = pickle.load(open("/media/data/BIAS/microbiome_GCN/AGES/MATRICES_AGES.pickle", "rb"))['TreeDistMatrix']

## Next steps

Now, we have our labels (the class labels corresponding to each sample ID), and the abundance of each microbe. So we are at the point where we start feature engineering.

1. we will need to normalize the abundance: you can divide each value of the abundance table by the total sum of microbes observed in that sample.
2. The order within each abundance vector will not matter for some methods, but will matter for others. For methods for which it matters, you will want to reorder the rows of the table via hierarchical clustering, using the distances in the distance matrix. The order should be fixed for all vectors.
3. We will need to convert the two dataframes into machine-learning compatible data types, so make sure the labels are in the same order as the columns of the abundance table, then convert to a dataset of X and Y, where X is a list of vectors, and Y is a vector where each index matches an index of X. You will probably need to convert the class labels to integers.
4. Then I guess we could start by trying to see if there is a signal with a very basic method, random forests, using the scikit-learn library ?
5. Then, we could try XGBoost (using the XGBoost Library).

For each, try many parameters to see if you get any significant signal (significantly more than 20%

Then we meet to see what results we got

We actually do not end up using XGBoost in the end, as it and support vector machines are significantly less accurate than random forest.

First, we define a dendrogram function to draw our clusters later

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = numpy.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = numpy.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

We then then use hierarchical clustering to see the number of clusters of features.

In [ ]:
n = len(dist_dict)
dist_mat = numpy.zeros(shape=(n, n))

for i, list_of_dists in enumerate(dist_dict.values()):
    for j, dists in enumerate(list_of_dists.values()):
        dist_mat[i][j] = dists

hierarchical_clustering = AgglomerativeClustering(distance_threshold=0, n_clusters=None, metric='precomputed', linkage='average')
hierarchical_clustering.fit(dist_mat)

plt.title("Dendrogram of clusters")
# plot the top three levels of the dendrogram
plot_dendrogram(hierarchical_clustering, truncate_mode="level", p=4)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

It appears that a good number of nodes would be around 16\
Using this, we bin each feature into a cluster as to reorder the features.

In [ ]:
num_nodes = 16
hierarchical_clustering = AgglomerativeClustering(n_clusters=num_nodes, metric='precomputed', linkage='average')
hierarchical_clustering.fit(dist_mat)
hierarchical_clustering.labels_

clusters = [[] for _ in range(num_nodes)]

# send the ith term to their respective cluster
for index, cluster in enumerate(hierarchical_clustering.labels_):
    clusters[cluster].append(features[index])
new_order = []
for cluster in clusters:
    new_order = new_order + cluster

Now we reorder the features, then we take the transpose of df_AGES to get the features to be horizontal.

In [ ]:
ordered_ages_df = df_AGES.loc[new_order].T
ordered_ages_df
# X = df_AGES_percentage.loc[selected_features].T
# X 

It appears there are some samples that have zeroes across all features, we remove them.

In [ ]:
ordered_ages_df = ordered_ages_df.loc[~((ordered_ages_df == 0).all(axis='columns'))]
labels = labels[ordered_ages_df.index]

now, we normalize each row into a percentage.

In [ ]:
sum = ordered_ages_df.sum(axis='columns')
ages_percentage_df = (ordered_ages_df.div(sum, axis='index'))
ages_percentage_df

Then we select features.\
Idea behind the criterion below is that we want to find features where there is a large difference between an age's average value for that feature and other ages' features. \
Standard deviation is used because mean may yield small values despite having large differences.

In [ ]:
ENCODINGS = {x : i for i, x in enumerate(sorted(ages_percentage_df.keys()))}
ENCODINGS
relabeled = ages_percentage_df.rename(mapper=ENCODINGS, axis='columns')

selected_features = set()
for i, age in enumerate(labels.unique()):
    own_deviation = relabeled.loc[labels[labels == age].index].std()
    other_deviation = relabeled.loc[labels[labels != age].index].std()
    evaluation = (own_deviation - other_deviation) ** 2
    selected_features |= set(list(evaluation.sort_values(ascending=False).iloc[0:700].index))

selected_features = list(selected_features)

selected_ages_df = relabeled.loc[:, selected_features]
selected_ages_df

Now we prepare our predictors 'X' and our labels 'y', using a test/train split of 0.1

In [ ]:
AGE_ENCODINGS = {x : i for i, x in enumerate(sorted(labels.unique()))}
y = labels.map(AGE_ENCODINGS).reindex(selected_ages_df.index)

X = selected_ages_df.to_numpy()
y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10 , random_state=11)

Training the model

In [ ]:
random_forest = RandomForestClassifier(random_state=11, n_estimators=200)
random_forest.fit(X_train, y_train)
random_forest.score(X_test, y_test)


Here is the lengthiest part, we use permutation importance to find the most significant predictor of each age category.

In [ ]:
from sklearn.inspection import permutation_importance

results = {}
for cls in AGE_ENCODINGS.values():
    mask = y_train == cls
    imp = permutation_importance(random_forest, X_train[mask], y_train[mask], n_repeats=1)
    results[cls] = imp.importances_mean

Then convert it back into a format that's readable to humans

In [ ]:
inv_encodings = {v: k for k, v in ENCODINGS.items()}
importance = {}
for age in results:
    temp = results[age]
    temp = {inv_encodings[i]: imp for i, imp in enumerate(temp)}
    temp = dict(sorted(temp.items(), key=lambda item: item[1], reverse=True))
    importance[sorted(labels.unique())[age]] = temp

with open("./feature_importance.pkl", "wb") as file:
    pickle.dump(importance, file, pickle.HIGHEST_PROTOCOL)

We can then load the importance matrix 

In [ ]:
with open('./feature_importance.pkl', 'rb') as file:
    importance = pickle.load(file)
importance

Importance contains how important each OTU is for determining the age category of the person, positive means people tend to be more likely to be the category if they have higher values of that OTU. Negative means people are less likely to be in category for higher values.